In [2]:
pip install pyspark

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.4/317.4 MB 1.6 MB/s eta 0:00:0000:0100:05
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.7-py2.py3-none-any.whl size=317907716 sha256=c99944ff62b9bb73bcb9b9d9880c09b2c77b7f7486ff8d57f5090e9a42c90a0b
  Stored in directory: /opt/work/.cache/pip/wheels/90/7b/3a/f4e0b629a2ec0ac23c6ea92ee30d7fb269def367fd9f70bcf9
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [8]:
try:
    spark.stop()
except Exception:
    pass

from pyspark.sql import SparkSession

spark = (
    SparkSession.builder.appName("Iceberg via REST")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    # define REST catalog named "rest"
    .config("spark.sql.catalog.rest", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.rest.type", "rest")
    .config("spark.sql.catalog.rest.uri", "http://iceberg-rest:8181")
    .config("spark.sql.catalog.rest.warehouse", "s3://lake/warehouse")
    .config("spark.sql.catalog.rest.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.rest.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.rest.s3.path-style-access", "true")
    .config("spark.sql.catalog.rest.s3.access-key-id", "admin")
    .config("spark.sql.catalog.rest.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.rest.s3.region", "us-east-1")
    .getOrCreate()
)

spark

25/11/05 21:41:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/11/05 21:41:42 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [9]:
spark.sql("SHOW NAMESPACES IN rest").show(truncate=False)

+---------+
|namespace|
+---------+
|raw      |
|silver   |
+---------+



In [10]:
spark.sql("SHOW TABLES IN rest.raw").show(truncate=False)

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|raw      |avito    |false      |
|raw      |mubawab  |false      |
+---------+---------+-----------+



In [11]:
spark.sql("SHOW TABLES IN rest.silver").show(truncate=False)

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|silver   |avito    |false      |
|silver   |mubawab  |false      |
+---------+---------+-----------+



In [12]:
tbl = "rest.silver.avito"
spark.sql(f"SELECT COUNT(*) AS rows FROM {tbl}").show()

+----+
|rows|
+----+
|  38|
+----+



In [13]:
df = spark.table("rest.silver.avito")

In [15]:
import pandas as pd
from IPython.display import display

pd.set_option("display.max_columns", 20)
pd.set_option("display.max_colwidth", 80)

pdf = (
    df
    .limit(10)
    .toPandas()
    .replace({r'[\r\n\t]+': ' '}, regex=True)
)


display(
    pdf.style
      .set_table_styles([
          {'selector': 'table', 'props': [('table-layout','fixed'), ('width','100%')]},
          {'selector': 'th, td', 'props': [
              ('max-width','280px'),
              ('white-space','nowrap'),
              ('overflow','hidden'),
              ('text-overflow','ellipsis')
          ]}
      ])
      .hide(axis='index')
)

print("Total rows (silver):", df.count())

25/11/05 21:42:08 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

id,url,title,price,description,seller_name,seller_type,image_urls,equipments,ingest_ts,offer,property_type,city,neighborhood,site,published_date,living_area,deposit,zoning,standing,total_area,floor,property_age,bathrooms,rooms,bedrooms,hoa_fee_per_month,condition,floors,availability,living_rooms,apartment_type
53079995,https://www.avito.ma/fr/hay_mohammadi/bureaux/Bureaux_à_louer_à_Casablanca_53079995.htm,Bureaux à louer à Casablanca,0.000000,Plateau bureau Neuf à louer pour usage professionnel sur Boulevard de la grande ceinture Emplacement - à 1 minute de la sortie de l’autoroute express - à 5 minutes d'Aïn Sebaâ et en face à préfecture mowahidin sur le boulevard principal - Bureaux lumineux et spacieux - Un immeuble imposant de trois étages doté d'une façade moderne Equipement - Pré-installation de la climatisation - Câblage informatique installé - Équipement téléphonique - Caméras - Ascenseurs - l’immeuble se compose d’une cour arrière qui aura accès à partir des bureaux arrière Contactez-nous pour une visite sur place,Plateaux bureaux Al-hizam,particulier,"['https://content.avito.ma/classifieds/images/10140667343?t=images', 'https://content.avito.ma/classifieds/images/10140668229?t=images', 'https://content.avito.ma/classifieds/images/10098872001?t=images', 'https://content.avito.ma/classifieds/images/10098872000?t=images', 'https://content.avito.ma/classifieds/images/10098871991?t=images', 'https://content.avito.ma/classifieds/images/10098872075?t=images', 'https://content.avito.ma/classifieds/images/10136234190?t=images', 'https://content.avito.ma/classifieds/images/10098872990?t=images', 'https://content.avito.ma/classifieds/images/10098871993?t=images', 'https://content.avito.ma/classifieds/images/10098872004?t=images', 'https://content.avito.ma/classifieds/images/10098871995?t=images']","['Ascenseur', 'Climatisation', 'Câblage téléphonique', 'Sécurité', 'Terrasse']",2025-11-04 21:50:10.006000,rent,Bureaux,Casablanca,Hay Mohammadi,avito,2025-11-04 14:50:08,None,None,None,None,None,1,None,None,1,None,None,None,None,None,None,None
55872849,https://www.avito.ma/fr/bouskoura_centre/bureaux/bureau_ou_cabinet_en_rez_de_chaussée_47m_à_Bouskoura_55872849.htm,bureau ou cabinet en rez-de-chaussée 47m à Bouskoura,6500.000000,"Je mets en location un local idéal pour siège bureau ou cabinet médical ou autre situé au centre de Bouskoura, proche de la gare du train, d'une superficie de 47m² avec 5m de hauteur, possibilités d'une mezzanine. Pour plus d'informations, veuillez me contacter. Intermédiaires s'abstenir.",abdelghani,particulier,None,"['Parking', 'Sécurité']",2025-11-04 21:50:20.007000,rent,Bureaux,Bouskoura,Bouskoura centre,avito,2025-11-04 22:46:14,None,None,None,None,47,None,None,2,None,None,None,None,None,None,None,None
56440843,https://www.avito.ma/fr/hay_riad/bureaux/Villa_à_usage_bureau_à_louer_Av_Abderrahim_Bouabid_56440843.htm,Villa à usage bureau à louer Av Abderrahim Bouabid,65000.000000,"À louer, une spacieuse villa à usage bureau située sur l’Avenue Abderrahim Bouabid. Superficie : 600 m² Deux façades offrant une excellente visibilité Idéale pour entreprises, cabinets ou sièges administratifs Emplacement stratégique dans un quartier professionnel et accessible Loyer mensuel : 65 000 DH Pour plus d’informations ou pour planifier une visite, veuillez contacter abdelhak",Agence Aboutaleb Immobilière,particulier,"['https://content.avito.ma/classifieds/images/10134759824?t=images', 'https://content.avito.ma/classifieds/images/10134759820?t=images', 'https://content.avito.ma/classifieds/images/10134759821?t=images', 'https://content.avito.ma/classifieds/images/10134759825?t=images', 'https://content.avito.ma/classifieds/images/10134759826?t=images', 'https://content.avito.ma/classifieds/images/10134759827?t=images', 'https://content.avito.ma/classifieds/images/10134759829?t=images']","['Climatisation', 'Câblage téléphonique', 'Parking', 'Sécurité', 'Terrasse']",2025-11-04 21:50:10.006000,rent,Bureaux,Rabat,Hay Riad,a

Total rows (silver): 38
